# GEORREFERENCIAMENTO OVP-DH
## Jupyter Notebook compartilhado

In [253]:
import numpy as np
import pandas as pd
import datetime as dt
import folium
from folium import plugins
from ipywidgets import interact, SelectionRangeSlider

### Abrindo dataset

In [254]:
#abrindo o arquivo csv
df = pd.read_csv('../data/data_geo_bo2016.csv').drop(columns=['Unnamed: 0'])
#lendo as cinco primeiras linhas
df.head()

,NUM_BO,RUBRICA,ANO,MES,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE
0,943,"Furto qualificado (art. 155, §4o.)",2016,1,S.PAULO,RUA JOAO MOURA,500,-23.560901,-46.677317
1,992386,Roubo (art. 157),2016,7,S.ANDRE,RUA MAURITI,60,-23.674312,-46.504567
2,1044074,Furto (art. 155),2016,8,S.PAULO,AVENIDA PRESIDENTE WILSON,4800,-23.594249,-46.589926
3,1996,Drogas sem autorização ou em desacordo (Art.33...,2016,3,S.PAULO,RUA DARIO VELOSO,130,-23.477746,-46.746526
4,815370,Roubo (art. 157),2016,6,S.PAULO,RUA MARIO COVAS,0,-23.505120,-46.367294


## Construindo o mapa 

### Definindo 'RUBRICA' como objetivo de pesquisa

In [255]:
print('Numero de rubricas: ', df['RUBRICA'].nunique(), '\n')
print('Descricao das rubricas: \n', df['RUBRICA'].unique())

Numero de rubricas:  13 

Descricao das rubricas: 
 ['Furto qualificado (art. 155, §4o.)' 'Roubo (art. 157)'
 'Furto (art. 155)'
 'Drogas sem autorização ou em desacordo (Art.33, caput)'
 'Lesão corporal (art 129 § 9º)' 'Lesão corporal (art. 129)'
 'Lesão corporal culposa na direção de veículo automotor (Art. 303)'
 'A.I.-Estupro de vulneravel (art.217-A)' 'A.I.-Roubo (art. 157)'
 'Estupro de vulneravel (art.217-A)'
 'Homicídio culposo na direção de veículo automotor (Art. 302)'
 'A.I.-Drogas sem autorização ou em desacordo (Art.33, caput)'
 'A.I.-Lesão corporal (art. 129)']


### Reformulando endereco, crime, lei, e data

In [256]:
# verifinando dados nulos em latitude e longitude

print(df.isna().sum())

NUM_BO               0
RUBRICA              0
ANO                  0
MES                  0
CIDADE               0
LOGRADOURO           0
NUMERO_LOGRADOURO    0
LATITUDE             0
LONGITUDE            0
dtype: int64


In [257]:
# criando endereco
df['ENDERECO'] = df['CIDADE'].str.strip() + ', ' + df['LOGRADOURO'].str.strip()

df.ENDERECO = df.ENDERECO.str.title()

In [258]:
# obtendo o crime constatado na rubrica
df[['CRIME','ARTIGO']] = df['RUBRICA'].str.split('(', expand=True)

In [259]:
# limpando os atributos criados
df.CRIME = df.CRIME.str.strip()
df.ARTIGO = df.ARTIGO.str.strip(')')

# observando a coluna de crimes
df.CRIME.unique()

array(['Furto qualificado', 'Roubo', 'Furto',
       'Drogas sem autorização ou em desacordo', 'Lesão corporal',
       'Lesão corporal culposa na direção de veículo automotor',
       'A.I.-Estupro de vulneravel', 'A.I.-Roubo',
       'Estupro de vulneravel',
       'Homicídio culposo na direção de veículo automotor',
       'A.I.-Drogas sem autorização ou em desacordo',
       'A.I.-Lesão corporal'], dtype=object)

In [260]:
# observando a coluna de leis
# a formatacao nao esta perfeita, mas nao ha artigos iguais com diferentes referencias
df.ARTIGO.unique()

array(['art. 155, §4o.', 'art. 157', 'art. 155', 'Art.33, caput',
       'art 129 § 9º', 'art. 129', 'Art. 303', 'art.217-A', 'Art. 302'],
      dtype=object)

In [261]:
# funcao para declarar os artigos homogeneamente
def pretifyartigo(artigo):
    artigo = artigo.capitalize().strip().strip('.')
    if "." in artigo:
        if ". " in artigo:
            return artigo
        else:
            i, j = artigo.split('.')
            artigo = i + '. ' + j
            return artigo
    else:
        i, j = artigo.split('t')
        artigo = i + 't. ' + j.strip()
        return artigo

In [262]:
# aplicando a funcao a coluna de artigos
df.ARTIGO = df.ARTIGO.apply(pretifyartigo)

In [263]:
# criando coluna de datetime
df["DATA"] = df.ANO.astype(str) + "-" + df.MES.astype(str)

df["DATA"] = pd.to_datetime(df.DATA, format="%Y-%m", dayfirst=True)

df.head()

,NUM_BO,RUBRICA,ANO,MES,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO,LATITUDE,LONGITUDE,ENDERECO,CRIME,ARTIGO,DATA
0,943,"Furto qualificado (art. 155, §4o.)",2016,1,S.PAULO,RUA JOAO MOURA,500,-23.560901,-46.677317,"S.Paulo, Rua Joao Moura",Furto qualificado,"Art. 155, §4o",2016-01-01
1,992386,Roubo (art. 157),2016,7,S.ANDRE,RUA MAURITI,60,-23.674312,-46.504567,"S.Andre, Rua Mauriti",Roubo,Art. 157,2016-07-01
2,1044074,Furto (art. 155),2016,8,S.PAULO,AVENIDA PRESIDENTE WILSON,4800,-23.594249,-46.589926,"S.Paulo, Avenida Presidente Wilson",Furto,Art. 155,2016-08-01
3,1996,Drogas sem autorização ou em desacordo (Art.33...,2016,3,S.PAULO,RUA DARIO VELOSO,130,-23.477746,-46.746526,"S.Paulo, Rua Dario Veloso",Drogas sem autorização ou em desacordo,"Art. 33, caput",2016-03-01
4,815370,Roubo (art. 157),2016,6,S.PAULO,RUA MARIO COVAS,0,-23.505120,-46.367294,"S.Paulo, Rua Mario Covas",Roubo,Art. 157,2016-06-01


### Definindo as cores dos marcadores e template do popup

In [264]:
### Definindo as cores

dic_nl = {'Furto qualificado':'lightgreen',
          'Furto':'lightgreen',
          'Roubo':'darkgreen',
          'A.I.-Roubo':'darkgreen',
          'Drogas sem autorização ou em desacordo':'purple',
          'A.I.-Drogas sem autorização ou em desacordo':'purple',
          'Lesão corporal':'black',
          'A.I.-Lesão corporal':'black',
          'Lesão corporal culposa na direção de veículo automotor':'cadetblue',
          'Homicídio culposo na direção de veículo automotor':'darkblue',
          'A.I.-Estupro de vulneravel':'darkred',
          'Estupro de vulneravel':'darkred',
         }

html_temp = """
    <h5>CRIME: </h5>
    {}
    <h5>ARTIGO: </h5>
    {}
    <h5>LOCAL: </h5>
    {}
    """

### Plotando sub-grupos em camadas

In [276]:
latmean = df['LATITUDE'].mean()
lonmean = df['LONGITUDE'].mean()
    
def plotmonth(month):
    data = df[df.DATA.dt.strftime('%b') == month]
    mapa_slider = folium.Map(location=[latmean,lonmean],
                             zoom_start=9,
                             control_scale=True)
    
    # Minimapa
    minimap = plugins.MiniMap(toggle_display=True)
    mapa_slider.add_child(minimap)

    # Plugin: fullscreen
    plugins.Fullscreen(position='topleft').add_to(mapa_slider)
    
    for bo in data.itertuples():
        crime = bo.CRIME
        artigo = bo.ARTIGO
        endereco = bo.ENDERECO 
        html = html_temp.format(crime, artigo, endereco)
        show_this = folium.Popup(html, max_width=280, min_width=250)
        folium.Marker(location=[bo.LATITUDE,bo.LONGITUDE],
                      popup=show_this,
                      icon=folium.Icon(icon='bolt', 
                                       color= dic_nl[crime],
                                       prefix='fa')).add_to(mapa_slider)
        
    return mapa_slider

### Visualizando o mapa

In [277]:
datas = [dt.date(2016,i,1) for i in range(1,13)]
options = [i.strftime('%b') for i in datas]

interact(plotmonth,
         month=SelectionSlider(options=options,
                               value=options[0],
                               description='Mes (2016)',
                               continuous_update=False,
                              ));

interactive(children=(SelectionSlider(continuous_update=False, description='Mes (2016)', options=('Jan', 'Feb'…